 Import the dataset, **Sleep_health_and_lifestyle_dataset.csv**.

In [ ]:
import pandas as pd

df = pd.read_csv('Sleep_health_and_lifestyle_dataset.csv')

Inspecting the dataset with **DataFrame#head**

In [ ]:
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


After inspecting the raw data, confirm the data type with **DataFrame#info**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           155 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


Then, using **DataFrame#describe** to calculate and display statistic information to the output cell.

In [ ]:
df.describe()

,Person ID,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps
count,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000
mean,187.500000,42.184492,7.132086,7.312834,59.171123,5.385027,70.165775,6816.844920
std,108.108742,8.673133,0.795657,1.196956,20.830804,1.774526,4.135676,1617.915679
min,1.000000,27.000000,5.800000,4.000000,30.000000,3.000000,65.000000,3000.000000
25%,94.250000,35.250000,6.400000,6.000000,45.000000,4.000000,68.000000,5600.000000
50%,187.500000,43.000000,7.200000,7.000000,60.000000,5.000000,70.000000,7000.000000
75%,280.750000,50.000000,7.800000,8.000000,75.000000,7.000000,72.000000,8000.000000
max,374.000000,59.000000,8.500000,9.000000,90.000000,8.000000,86.000000,10000.000000


# Data Preparation

After inspecting the data, we first check for null data in each columns. The result is shown as expected. Every columns contain non-null data excepted for Sleep Disorder, because null mean doesn't have any sleep disorder.

In [ ]:
df.isnull().sum()

,0
Person ID,0
Gender,0
Age,0
Occupation,0
Sleep Duration,0
Quality of Sleep,0
Physical Activity Level,0
Stress Level,0
BMI Category,0
Blood Pressure,0


The data doesn't contain duplicated thanks to **Person ID** that act as a Unique ID.

In [ ]:
df.duplicated().sum()

np.int64(0)

To make the data is more convinence for us to process further, we remove a leading and trailing space, make it a lower case, and replace the space between words with underscore(_)

In [ ]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

Inspecting each category to make surre the category doesn't duplicate the meaning.

In [ ]:
df['gender'].unique()

array(['Male', 'Female'], dtype=object)

In [ ]:
df['occupation'].unique()

array(['Software Engineer', 'Doctor', 'Sales Representative', 'Teacher',
       'Nurse', 'Engineer', 'Accountant', 'Scientist', 'Lawyer',
       'Salesperson', 'Manager'], dtype=object)

In [ ]:
df['bmi_category'].unique()

array(['Overweight', 'Normal', 'Obese', 'Normal Weight'], dtype=object)

In [ ]:
df['sleep_disorder'].unique()

array([nan, 'Sleep Apnea', 'Insomnia'], dtype=object)

After inspecting each category, we can see that in bmi_category the *Normal* and *Normal Weight* have the same meaning. So we replace *Normal Weight* with *Normal*.

In [ ]:
df['bmi_category'] = df["bmi_category"].replace("Normal Weight", "Normal")

Then, we replace the *null* data in **sleep_disorder** with *Normal*

In [ ]:
df['sleep_disorder'] = df['sleep_disorder'].fillna("Normal")

Lastly, We seperate the **blood_pressure** to a seperate colums, **systolic**, and **diastolic**.

In [ ]:
df[['systolic', 'diastolic']] = df['blood_pressure'].str.split('/', expand=True).astype(int)

df.drop('blood_pressure', axis=1, inplace=True)

## Feature Engineering

We categorize the **age**, **stress_level**, and **physical_activity__level** to make it convinice to perform the analysis.

In [ ]:
bins = [0, 18, 30, 45, 60, 100]
labels = ['Teen', 'Young Adult', 'Adult', 'Middle-aged', 'Senior']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels)

In [ ]:
stress_bins = [0, 3, 6, 10]
stress_labels = ['Low', 'Moderate', 'High']
df['stress_category'] = pd.cut(df['stress_level'], bins=stress_bins, labels=stress_labels)

In [ ]:
bins = [29, 44, 59, 74, 90]
labels = ['Sedentary', 'Light', 'Moderate', 'Active']
df['activity_category'] = pd.cut(df['physical_activity_level'], bins=bins, labels=labels)

Create a column, to check if the person is sleep more than 7 hours.

In [ ]:
df['sufficient_sleep'] = (df['sleep_duration'] >= 7).astype(int)

Calculate activity and sleep ratio showing who is under-sleeping or overworking.

In [ ]:
df['activity_sleep_ratio'] = df['physical_activity_level'] / df['sleep_duration']

Check for null to confirm the new columns we engineered doesn't go wrong.

In [ ]:
df.isnull().sum()

,0
person_id,0
gender,0
age,0
occupation,0
sleep_duration,0
quality_of_sleep,0
physical_activity_level,0
stress_level,0
bmi_category,0
heart_rate,0


Inspecting the data after preparation.

In [ ]:
df.head()

,person_id,gender,age,occupation,sleep_duration,quality_of_sleep,physical_activity_level,stress_level,bmi_category,heart_rate,daily_steps,sleep_disorder,systolic,diastolic,age_group,stress_category,activity_category,sufficient_sleep,activity_sleep_ratio
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,77,4200,Normal,126,83,Young Adult,Moderate,Sedentary,0,6.885246
1,2,Male,28,Doctor,6.2,6,60,8,Normal,75,10000,Normal,125,80,Young Adult,High,Moderate,0,9.677419
2,3,Male,28,Doctor,6.2,6,60,8,Normal,75,10000,Normal,125,80,Young Adult,High,Moderate,0,9.677419
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,85,3000,Sleep Apnea,140,90,Young Adult,High,Sedentary,0,5.084746
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,85,3000,Sleep Apnea,140,90,Young Adult,High,Sedentary,0,5.084746


Save to a csv file, **Sleep_health_and_lifestyle_prepared.csv**.

In [ ]:
df.to_csv("Sleep_health_and_lifestyle_prepared.csv", index=False)

# EDA

## Calculate correlation

Extract the columns with number and display to the output cell.

In [ ]:
numeric_df = df.select_dtypes(include='number')

numeric_df.head()

,person_id,age,sleep_duration,quality_of_sleep,physical_activity_level,stress_level,heart_rate,daily_steps,systolic,diastolic,sufficient_sleep,activity_sleep_ratio
0,1,27,6.1,6,42,6,77,4200,126,83,0,6.885246
1,2,28,6.2,6,60,8,75,10000,125,80,0,9.677419
2,3,28,6.2,6,60,8,75,10000,125,80,0,9.677419
3,4,28,5.9,4,30,8,85,3000,140,90,0,5.084746
4,5,28,5.9,4,30,8,85,3000,140,90,0,5.084746


Drop the **person_id** column, because we don't need it.

In [ ]:
numeric_df.drop(['person_id'], axis=1, inplace=True)

Inspecting the data to confirm the columns we wanted to calculate the correlation

In [ ]:
numeric_df.head()

,age,sleep_duration,quality_of_sleep,physical_activity_level,stress_level,heart_rate,daily_steps,systolic,diastolic,sufficient_sleep,activity_sleep_ratio
0,27,6.1,6,42,6,77,4200,126,83,0,6.885246
1,28,6.2,6,60,8,75,10000,125,80,0,9.677419
2,28,6.2,6,60,8,75,10000,125,80,0,9.677419
3,28,5.9,4,30,8,85,3000,140,90,0,5.084746
4,28,5.9,4,30,8,85,3000,140,90,0,5.084746


Calculate the correlation. Remove index and rename the column names and save to a csv file.

In [ ]:
corr = numeric_df.corr()

corr_long = corr.reset_index().melt(id_vars='index')
corr_long.columns = ['variable_1', 'variable_2', 'correlation']

corr_long.to_csv("correlation_tableau.csv", index=False)